In [1]:
# !pip install keras-tuner

In [14]:
import tensorflow
from tensorflow import keras
import numpy as np

In [15]:
fasion_mnist = keras.datasets.fashion_mnist

In [16]:
fasion_mnist

<module 'keras.api._v2.keras.datasets.fashion_mnist' from '/usr/local/lib/python3.8/dist-packages/keras/api/_v2/keras/datasets/fashion_mnist/__init__.py'>

In [17]:
(train_images,train_labels),(test_images,test_labels) = fasion_mnist.load_data()

In [18]:
train_images = train_images/255.0
test_images = test_images/255.0

In [19]:
train_images[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [20]:
train_images.shape

(60000, 28, 28)

In [21]:
train_images[0].shape

(28, 28)

In [22]:
len(train_images)

60000

In [23]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [24]:
train_images.shape

(60000, 28, 28, 1)

In [25]:
test_images.shape

(10000, 28, 28, 1)

In [59]:
def build_model(hp):
    model =keras.Sequential([keras.layers.Conv2D(filters = hp.Int('conv_1_filter',min_value =32, max_value =128, step =16),
                                               kernel_size =hp.Choice('conv_1_kernel', values =[3,5]),activation ='relu', input_shape =(28,28,1)),
                           
                           keras.layers.Conv2D(filters = hp.Int('conv_1_filter',min_value =32, max_value =128, step =16),
                                               kernel_size =hp.Choice('conv_1_kernel', values =[3,5]),activation ='relu'),
                           
                           keras.layers.Flatten(),

                           keras.layers.Dense(units =hp.Int('dense_1_units',min_value =32, max_value =128, step =16),activation= 'relu'),

                           keras.layers.Dense(units = 10, activation ='softmax')])
    model.compile(optimizer = keras.optimizers.Adam(hp.Choice("learning_rate",values =[1e-2,1e-3])),
                loss = 'sparse_categorical_crossentropy', metrics =['accuracy'])
    return model

In [56]:
# def build_model(hp):  
#   model = keras.Sequential([
#     keras.layers.Conv2D(
#         filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
#         kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
#         activation='relu',
#         input_shape=(28,28,1)
#     ),
#     keras.layers.Conv2D(
#         filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
#         kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
#         activation='relu'
#     ),
#     keras.layers.Flatten(),
#     keras.layers.Dense(
#         units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
#         activation='relu'
#     ),
#     keras.layers.Dense(10, activation='softmax')
#   ])
  
#   model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])
  
#   return model

In [60]:
import keras_tuner

from keras_tuner import RandomSearch
from keras_tuner import HyperParameters

In [61]:
tuner_search = RandomSearch(build_model,
                            objective= 'val_accuracy', 
                            max_trials= 5, directory ='output', project_name ='mnist fashion')

In [62]:
tuner_search.search(train_images, train_labels, epochs =3, validation_split =0.1)

Trial 5 Complete [00h 00m 33s]
val_accuracy: 0.8543333411216736

Best val_accuracy So Far: 0.903166651725769
Total elapsed time: 00h 03m 17s


In [63]:
model = tuner_search.get_best_models(num_models=1)[0]

In [65]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 32)        832       
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 20, 32)        25632     
                                                                 
 flatten (Flatten)           (None, 12800)             0         
                                                                 
 dense (Dense)               (None, 80)                1024080   
                                                                 
 dense_1 (Dense)             (None, 10)                810       
                                                                 
Total params: 1,051,354
Trainable params: 1,051,354
Non-trainable params: 0
_________________________________________________________________


In [66]:
model.fit(train_images, train_labels,epochs= 10, validation_split= 0.1, initial_epoch= 3)

Epoch 4/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1775 - accuracy: 0.9336 - val_loss: 0.2737 - val_accuracy: 0.9018
Epoch 5/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1423 - accuracy: 0.9461 - val_loss: 0.2719 - val_accuracy: 0.9117
Epoch 6/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1118 - accuracy: 0.9582 - val_loss: 0.3063 - val_accuracy: 0.9078
Epoch 7/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0868 - accuracy: 0.9676 - val_loss: 0.3276 - val_accuracy: 0.9057
Epoch 8/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0667 - accuracy: 0.9751 - val_loss: 0.3750 - val_accuracy: 0.9098
Epoch 9/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0544 - accuracy: 0.9801 - val_loss: 0.3871 - val_accuracy: 0.9112
Epoch 10/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0449 - accuracy: 0.9833 - val_loss: 0.4831 - val_accuracy